***
***
# 使用GraphLab进行电影推荐
***
***


In [1]:
import graphlab
graphlab.canvas.set_target("ipynb")
# set canvas to show sframes and sgraphs in ipython notebook
# import matplotlib.pyplot as plt
# %matplotlib inline

In [7]:
# download data from: http://files.grouplens.org/datasets/movielens/ml-1m.zip

In [2]:
data = graphlab.SFrame.read_csv('/Users/datalab/bigdata/cjc/ml-1m/ratings.dat', delimiter='\n', 
                                header=False)['X1'].apply(lambda x: x.split('::')).unpack()
for col in data.column_names():
    data[col] = data[col].astype(int)
data.rename({'X.0': 'user_id', 'X.1': 'movie_id', 'X.2': 'rating', 'X.3': 'timestamp'})
data.save('ratings')

users = graphlab.SFrame.read_csv('/Users/datalab/bigdata/cjc/ml-1m/users.dat', delimiter='\n', 
                                 header=False)['X1'].apply(lambda x: x.split('::')).unpack()
users.rename({'X.0': 'user_id', 'X.1': 'gender', 'X.2': 'age', 'X.3': 'occupation', 'X.4': 'zip-code'})
users['user_id'] = users['user_id'].astype(int)
users.save('users')

items = graphlab.SFrame.read_csv('/Users/datalab/bigdata/cjc/ml-1m/movies.dat', delimiter='\n', 
                                 header=False)['X1'].apply(lambda x: x.split('::')).unpack()
items.rename({'X.0': 'movie_id', 'X.1': 'title', 'X.2': 'genre'})
items['movie_id'] = items['movie_id'].astype(int)
items.save('items')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1525501840.log


This non-commercial license of GraphLab Create for academic use is assigned to wangchengjun@nju.edu.cn and will expire on March 14, 2019.


Finished parsing file /Users/datalab/bigdata/cjc/ml-1m/ratings.dat

Parsing completed. Parsed 100 lines in 0.43242 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Finished parsing file /Users/datalab/bigdata/cjc/ml-1m/movies.dat

Parsing completed. Parsed 100 lines in 0.033283 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/datalab/bigdata/cjc/ml-1m/movies.dat

Parsing completed. Parsed 3883 lines in 0.016475 secs.

In [3]:
data

user_id,movie_id,rating,timestamp
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
1,1197,3,978302268
1,1287,5,978302039
1,2804,5,978300719
1,594,4,978302268
1,919,4,978301368


In [4]:
items

movie_id,title,genre
1,Toy Story (1995),Animation|Children's|Comedy ...
2,Jumanji (1995),Adventure|Children's|Fantasy ...
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride PartII (1995) ...,Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children's
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
users

user_id,gender,age,occupation,zip-code
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117
4,M,45,7,02460
5,M,25,20,55455
6,F,50,9,55117
7,M,35,1,06810
8,M,25,12,11413
9,M,25,17,61614
10,F,35,1,95370


In [7]:
data = data.join(items, on='movie_id')

In [8]:
data

user_id,movie_id,rating,timestamp,title,genre
1,1193,5,978300760,One Flew Over theCuckoo's Nest (1975) ...,Drama
1,661,3,978302109,James and the Giant Peach(1996) ...,Animation|Children's|Musical ...
1,914,3,978301968,My Fair Lady (1964),Musical|Romance
1,3408,4,978300275,Erin Brockovich (2000),Drama
1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy ...
1,1197,3,978302268,"Princess Bride, The(1987) ...",Action|Adventure|Comedy|Romance ...
1,1287,5,978302039,Ben-Hur (1959),Action|Adventure|Drama
1,2804,5,978300719,"Christmas Story, A (1983)",Comedy|Drama
1,594,4,978302268,Snow White and the SevenDwarfs (1937) ...,Animation|Children's|Musical ...
1,919,4,978301368,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical ...


In [9]:
train_set, test_set = data.random_split(0.95, seed=1)

In [10]:
m = graphlab.recommender.create(train_set, 'user_id', 'movie_id', 'rating')

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 949852 observations with 6040 users and 3701 items.

Data prepared in: 1.29085s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.625427

Final training RMSE: 0.768104

In [11]:
m

Class                            : RankingFactorizationRecommender

Schema
------
User ID                          : user_id
Item ID                          : movie_id
Target                           : rating
Additional observation features  : 3
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 949852
Number of users                  : 6040
Number of items                  : 3701

Training summary
----------------
Training time                    : 33.7974

Model Parameters
----------------
Model class                      : RankingFactorizationRecommender
num_factors                      : 32
binary_target                    : 0
side_data_factorization          : 1
solver                           : auto
nmf                              : 0
max_iterations                   : 25

Regularization Settings
-----------------------
regularization                   : 0.0
regularization_type              : n

In [12]:
m2 = graphlab.item_similarity_recommender.create(train_set, 
                                                 'user_id', 'movie_id', 'rating',
                                 similarity_type='pearson')

Recsys training: model = item_similarity

Warning: Ignoring columns timestamp, title, genre;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 949852 observations with 6040 users and 3701 items.

...

| 3.17s                               | 100              | 3701            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.21611s

In [13]:
m2

Class                            : ItemSimilarityRecommender

Schema
------
User ID                          : user_id
Item ID                          : movie_id
Target                           : rating
Additional observation features  : 0
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 949852
Number of users                  : 6040
Number of items                  : 3701

Training summary
----------------
Training time                    : 3.2161

Model Parameters
----------------
Model class                      : ItemSimilarityRecommender
threshold                        : 0.001
similarity_type                  : pearson
training_method                  : auto

Other Settings
--------------
degree_approximation_threshold   : 4096
sparse_density_estimation_sample_size : 4096
max_data_passes                  : 4096
target_memory_usage              : 8589934592
seed_item_set_size               : 50


In [14]:
result = graphlab.recommender.util.compare_models(test_set, 
                                                  [m, m2],
                                            user_sample=.5, skip_set=train_set)

compare_models: using 2811 users to estimate model performance
PROGRESS: Evaluate model M0


recommendations finished on 1000/2811 queries. users per second: 7788.34

recommendations finished on 2000/2811 queries. users per second: 8209.17


Precision and recall summary statistics by cutoff
+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    | 0.0647456421202 | 0.0055003280382 |
|   2    | 0.0585200996087 | 0.0113353035287 |
|   3    | 0.0524131388592 | 0.0159704348141 |
|   4    | 0.0489149768766 | 0.0202725808447 |
|   5    | 0.0452508004269 | 0.0240583282714 |
|   6    | 0.0432230522946 | 0.0281284957357 |
|   7    | 0.0413680947299 | 0.0315008447446 |
|   8    |  0.03962113127  | 0.0342475240871 |
|   9    | 0.0383414364204 | 0.0373489084669 |
|   10   | 0.0367484880825 | 0.0397919699688 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 0.908462581230655)

Per User RMSE (best)
+---------+-------+------------------+
| user_id | count |       rmse       |
+---------+-------+------------------+
|   1614  |   1   | 0.00313141188165 |
+---------+-------+------------------+
[1 rows x 3

recommendations finished on 1000/2811 queries. users per second: 21419.2

recommendations finished on 2000/2811 queries. users per second: 22703.3


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    |        0.0        |        0.0        |
|   3    |        0.0        |        0.0        |
|   4    | 8.89363215937e-05 | 5.08207551964e-05 |
|   5    |  7.1149057275e-05 | 5.08207551964e-05 |
|   6    | 5.92908810625e-05 | 5.08207551964e-05 |
|   7    | 5.08207551964e-05 | 5.08207551964e-05 |
|   8    | 4.44681607969e-05 | 5.08207551964e-05 |
|   9    | 3.95272540417e-05 | 5.08207551964e-05 |
|   10   | 3.55745286375e-05 | 5.08207551964e-05 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 0.9853701447799256)

Per User RMSE (best)
+---------+-------+------------------+
| user_id | count |       rmse       |
+---------+-------+------------------+
|   5821  |   1   | 0.0049398055

### Getting similar items

In [15]:
m.get_similar_items([1287])  # movie_id is Ben-Hur

movie_id,similar,score,rank
1287,940,0.62611323595,1
1287,1291,0.578746318817,2
1287,2370,0.560160756111,3
1287,3805,0.524255812168,4
1287,2905,0.523368954659,5
1287,2324,0.519006967545,6
1287,2804,0.510964155197,7
1287,1214,0.496943891048,8
1287,919,0.491189420223,9
1287,1198,0.490789085627,10


In [19]:
help(m.get_similar_items)

Help on method get_similar_items in module graphlab.toolkits.recommender.util:

get_similar_items(self, items=None, k=10, verbose=False) method of graphlab.toolkits.recommender.ranking_factorization_recommender.RankingFactorizationRecommender instance
    Get the k most similar items for each item in items.
    
    Each type of recommender has its own model for the similarity
    between items. For example, the item_similarity_recommender will
    return the most similar items according to the user-chosen
    similarity; the factorization_recommender will return the
    nearest items based on the cosine similarity between latent item
    factors.
    
    Parameters
    ----------
    items : SArray or list; optional
        An :class:`~graphlab.SArray` or list of item ids for which to get
        similar items. If 'None', then return the `k` most similar items for
        all items in the training set.
    
    k : int, optional
        The number of similar items for each item.
    

'score' gives the similarity score of that item

In [16]:
m.get_similar_items([1287]).join(items, on={'similar': 'movie_id'}).sort('rank')

movie_id,similar,score,rank,title,genre
1287,940,0.62611323595,1,"Adventures of Robin Hood,The (1938) ...",Action|Adventure
1287,1291,0.578746318817,2,Indiana Jones and theLast Crusade (1989) ...,Action|Adventure
1287,2370,0.560160756111,3,"Emerald Forest, The(1985) ...",Action|Adventure|Drama
1287,3805,0.524255812168,4,Knightriders (1981),Action|Adventure|Drama
1287,2905,0.523368954659,5,Sanjuro (1962),Action|Adventure
1287,2324,0.519006967545,6,Life Is Beautiful (LaVita � bella) (1997) ...,Comedy|Drama
1287,2804,0.510964155197,7,"Christmas Story, A (1983)",Comedy|Drama
1287,1214,0.496943891048,8,Alien (1979),Action|Horror|Sci-Fi|Thriller ...
1287,919,0.491189420223,9,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical ...
1287,1198,0.490789085627,10,Raiders of the Lost Ark(1981) ...,Action|Adventure


### Making recommendations

In [20]:
recs = m.recommend()

recommendations finished on 1000/6040 queries. users per second: 7665.01

recommendations finished on 2000/6040 queries. users per second: 7859.16

recommendations finished on 3000/6040 queries. users per second: 7905.28

recommendations finished on 4000/6040 queries. users per second: 8137.29

recommendations finished on 5000/6040 queries. users per second: 8219.12

recommendations finished on 6000/6040 queries. users per second: 7955.95

In [21]:
recs

user_id,movie_id,score,rank
1,34,3.9719229951,1
1,590,3.92724250028,2
1,1198,3.91728875557,3
1,1282,3.91183034102,4
1,1682,3.90261084358,5
1,356,3.89610960106,6
1,1408,3.88136133323,7
1,1210,3.87818231205,8
1,912,3.87265440965,9
1,1393,3.87153578973,10


In [24]:
data[data['user_id'] == 4]

user_id,movie_id,rating,timestamp,title,genre
4,3468,5,978294008,"Hustler, The (1961)",Drama
4,1210,3,978293924,Star Wars: Episode VI -Return of the Jedi (1 ...,Action|Adventure|Romance|Sci-Fi|War ...
4,2951,4,978294282,"Fistful of Dollars, A(1964) ...",Action|Western
4,1214,4,978294260,Alien (1979),Action|Horror|Sci-Fi|Thriller ...
4,1036,4,978294282,Die Hard (1988),Action|Thriller
4,260,5,978294199,Star Wars: Episode IV - ANew Hope (1977) ...,Action|Adventure|Fantasy|Sci-Fi ...
4,2028,5,978294230,Saving Private Ryan(1998) ...,Action|Drama|War
4,480,4,978294008,Jurassic Park (1993),Action|Adventure|Sci-Fi
4,1196,2,978294199,Star Wars: Episode V -The Empire Strikes Back ...,Action|Adventure|Drama|Sci-Fi|War ...
4,1198,5,978294199,Raiders of the Lost Ark(1981) ...,Action|Adventure


In [25]:
m.recommend(users=[4], k=20).join(items, on='movie_id')

user_id,movie_id,score,rank,title,genre
4,527,4.10534794144,7,Schindler's List (1993),Drama|War
4,541,4.07768926285,9,Blade Runner (1982),Film-Noir|Sci-Fi
4,745,4.02218672428,17,"Close Shave, A (1995)",Animation|Comedy|Thriller
4,750,4.31101408861,1,Dr. Strangelove or: How ILearned to Stop Worrying ...,Sci-Fi|War
4,924,4.14404563866,5,2001: A Space Odyssey(1968) ...,Drama|Mystery|Sci-Fi|Thriller ...
4,1073,4.0288983833,15,Willy Wonka and theChocolate Factory (1971) ...,Adventure|Children's|Comedy|Fantasy ...
4,1084,4.01916591338,19,Bonnie and Clyde (1967),Crime|Drama
4,1094,4.07409318052,10,"Crying Game, The (1992)",Drama|Romance|War
4,1183,4.01506134383,20,"English Patient, The(1996) ...",Drama|Romance|War
4,1206,4.08579082442,8,"Clockwork Orange, A(1971) ...",Sci-Fi


In [26]:
m.recommend?

### Recommendations for new users

In [51]:
recent_data = graphlab.SFrame()
recent_data['movie_id'] = [30, 1000, 900, 883, 251, 200, 199, 180, 120, 991, 1212] 
recent_data['user_id'] = 99999
recent_data['rating'] = [2, 1, 3, 4, 0, 0, 1, 1, 1, 2, 3]
recent_data

movie_id,user_id,rating
30,99999,2
1000,99999,1
900,99999,3
883,99999,4
251,99999,0
200,99999,0
199,99999,1
180,99999,1
120,99999,1
991,99999,2


In [53]:
m2.recommend(users=[99999], new_observation_data=recent_data).join(items, on='movie_id').sort('rank')

user_id,movie_id,score,rank,title,genre
99999,572,5.0,1,Foreign Student (1994),Drama
99999,3881,5.0,2,Bittersweet Motel (2000),Documentary
99999,1830,5.0,3,Follow the Bitch (1998),Comedy
99999,989,5.0,4,Schlafes Bruder (Brotherof Sleep) (1995) ...,Drama
99999,3172,5.0,5,Ulysses (Ulisse) (1954),Adventure
99999,3233,5.0,6,Smashing Time (1967),Comedy
99999,3382,5.0,7,Song of Freedom (1936),Drama
99999,787,5.0,8,"Gate of Heavenly Peace,The (1995) ...",Documentary
99999,3656,5.0,9,Lured (1947),Crime
99999,3280,5.0,10,"Baby, The (1973)",Horror


### Saving and loading models

In [ ]:
m.save('my_model')

In [ ]:
m_again = graphlab.load_model('my_model')

In [ ]:
m_again